In [ ]:
import pandas as pd

In [ ]:
import gensim

In [ ]:
df = pd.read_csv('/content/train.txt',delimiter=';',names=['sentence','label'])

In [ ]:
dff = pd.read_csv('/content/test.txt',delimiter=';',names=['sentence','label'])

In [ ]:
df.shape

In [ ]:
dff.shape

In [ ]:
dff.head()

In [ ]:
dft=df.append(dff)

In [ ]:
df.head()

In [ ]:
dft.shape

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)
            
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
    
            
    return " ".join(y)

In [ ]:
dft['transformed_text']=dft['sentence'].apply(transform_text)
    

In [ ]:
dft['label'].value_counts()

In [ ]:
dft.head()

In [ ]:
data = dft[['transformed_text','label']]

In [ ]:
data.head()

In [ ]:
X = data.iloc[:12000,0:1]
y = data['label']
y = y.iloc[:12000]

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

# BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
X_train_bow = cv.fit_transform(X_train['transformed_text']).toarray()
X_test_bow = cv.transform(X_test['transformed_text']).toarray()

In [ ]:
X_train_bow.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

In [ ]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

In [ ]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['transformed_text']).toarray()
X_test_bow = cv.transform(X_test['transformed_text']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

In [ ]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['transformed_text']).toarray()
X_test_bow = cv.transform(X_test['transformed_text']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

# TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train['transformed_text']).toarray()
X_test_tfidf = tfidf.transform(X_test['transformed_text'])

In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

# Word2vec

In [ ]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
sent = [row.split() for row in X['transformed_text']]

In [ ]:
len(sent)

In [ ]:
# from gensim.models.phrases import Phrases, Phraser

In [ ]:
# phrases = Phrases(sent, min_count=30, progress_per=500)

In [ ]:
# bigram = Phraser(phrases)

In [ ]:
# sentences = bigram[sent]

In [ ]:
# sentences

In [ ]:
model = gensim.models.Word2Vec(
    sent,
    window=5,
    min_count=2
)

In [ ]:
model

In [ ]:
#  model.build_vocab(sent)

In [ ]:
# model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.corpus_count

In [ ]:
model.wv.similar_by_word('man')

In [ ]:
# model.wv.index_to_key

In [ ]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
c = []
for doc in tqdm(X['transformed_text'].values):
    c.append(document_vector(doc))

In [ ]:
C= np.array(c)

In [ ]:
C.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(C,y,test_size=0.2,random_state=1)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)